# Calculadora de avance

## Datos del proyecto

In [254]:
proyecto = "callacalla"
mes_en_analisis = 5
anio = 2025

## Librerias necesarias

In [255]:
from dotenv import load_dotenv, find_dotenv
from dotenv import load_dotenv, find_dotenv
from functools import reduce
from google.cloud import firestore
from google.oauth2 import service_account
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from pathlib import Path
from typing import Dict, List, Any
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import excel2img
import itertools
import json
import os
import pandas as pd
import pickle
import pprint
import re
import win32com.client as win32
import xlsxwriter

## Funciones utiles

### Fusionar diccionarios

In [256]:
def fusionar_diccionarios(diccionario_de_diccionarios):
    """
    Fusiona diccionarios separados en un diccionario unificado.
    Solo incluye claves donde ambos valores sean diferentes de cero.
    Usa dinámicamente las claves del diccionario de entrada.
    
    Args:
        diccionario_de_diccionarios (dict): Diccionario con estructura:
            {
                "nombre_campo1": {clave: valor, ...},
                "nombre_campo2": {clave: valor, ...}
            }
    
    Returns:
        dict: Diccionario fusionado con estructura {clave: {nombre_campo1: valor, nombre_campo2: valor}}
    """
    # Obtener las claves (nombres de los campos) del diccionario principal
    nombres_campos = list(diccionario_de_diccionarios.keys())
    
    if len(nombres_campos) != 2:
        raise ValueError("El diccionario debe contener exactamente 2 campos")
    
    campo1_nombre = nombres_campos[0]
    campo2_nombre = nombres_campos[1]
    
    campo1_datos = diccionario_de_diccionarios[campo1_nombre]
    campo2_datos = diccionario_de_diccionarios[campo2_nombre]
    
    fusionado = {}
    
    # Obtener todas las claves únicas de ambos diccionarios
    todas_las_claves = set(campo1_datos.keys()) | set(campo2_datos.keys())
    
    for clave in todas_las_claves:
        valor1 = campo1_datos.get(clave, 0)
        valor2 = campo2_datos.get(clave, 0)
        
        # Solo agregar si ambos valores son diferentes de cero
        if valor1 != 0 and valor2 != 0:
            fusionado[clave] = {
                campo1_nombre: valor1,
                campo2_nombre: valor2
            }
    
    return fusionado

### Calculadora de costo total

In [257]:
from typing import TypedDict


class CostoIngenieriaResult(TypedDict):
    costo_directo: float
    gastos_generales: float
    utilidad: float
    subtotal: float
    igv: float
    total: float


def calculadora_costo_total(costo_directo: float) -> CostoIngenieriaResult:
    """
    Calcula el costo total de un proyecto de ingeniería civil en soles peruanos.

    Args:
        costo_directo (float): Costo directo del proyecto

    Returns:
        CostoIngenieriaResult: Diccionario con todos los valores calculados
    """
    # Calcular gastos generales (10% del costo directo)
    gastos_generales = round(costo_directo * 0.10, 2)

    # Calcular utilidad (5% del costo directo)
    utilidad = round(costo_directo * 0.05, 2)

    # Calcular subtotal
    subtotal = round(costo_directo + gastos_generales + utilidad, 2)

    # Calcular IGV (18% del subtotal)
    igv = round(subtotal * 0.18, 2)

    # Calcular total
    total = round(subtotal + igv, 2)

    return {
        "costo_directo": round(costo_directo, 2),
        "gastos_generales": gastos_generales,
        "utilidad": utilidad,
        "subtotal": subtotal,
        "igv": igv,
        "total": total,
    }

### Multiplicador de precio unitario actualizado por cargas de trabajo

In [258]:
def calcular_costos_unitarios(
    dict_precios_unitarios_actualizados, cargas_trabajo
):
    """
    Multiplica cada carga de trabajo del contratista por su precio unitario
    correspondiente.

    Args:
        dict_precios_unitarios_actualizados (dict): mapea código → precio unitario.
        cargas_trabajo (dict): mapea código → carga de trabajo.

    Returns:
        dict: mapea código → precio total (precio unitario * carga).

    Raises:
        KeyError: si alguna clave de cargas_trabajo no existe en
                  dict_precios_unitarios_actualizados.
    """
    # Comprobar que no falte ninguna clave
    faltantes = set(cargas_trabajo) - set(
        dict_precios_unitarios_actualizados
    )
    if faltantes:
        raise KeyError(f"Faltan precios unitarios para las claves: {faltantes}")

    # Generar el diccionario resultado
    resultado = {
        clave: dict_precios_unitarios_actualizados[clave]
        * cargas_trabajo[clave]
        for clave in cargas_trabajo
    }
    return resultado

### Multiplicador de precio unitario por carga de trabajo v2

In [259]:
def calcular_costos_unitarios_v2(
    dict_precios_unitarios_actualizados, cargas_trabajo
):
    """
    Multiplica cada carga de trabajo del contratista por su precio unitario
    correspondiente.

    Si alguna clave en cargas_trabajo tiene carga > 0 y no existe
    en dict_precios_unitarios_actualizados, levanta KeyError.
    Claves con carga = 0 se ignoran (no dan error), y claves en precios que no
    están en cargas se tratan con carga = 0.

    Args:
        dict_precios_unitarios_actualizados (dict): código → precio unitario.
        cargas_trabajo (dict): código → carga de trabajo.

    Returns:
        dict: código → precio total (precio_unitario * carga).

    Raises:
        KeyError: si alguna clave con carga > 0 en cargas_trabajo
                  no existe en dict_precios_unitarios_actualizados.
    """
    # Detectar faltantes solo para cargas > 0
    faltantes = {
        codigo
        for codigo, carga in cargas_trabajo.items()
        if carga != 0 and codigo not in dict_precios_unitarios_actualizados
    }
    if faltantes:
        raise KeyError(f"Faltan precios unitarios para las claves: {faltantes}")

    # Construir resultado: para cada precio, multiplicar por carga (0 si no hay)
    resultado = {}
    for codigo, precio_unitario in dict_precios_unitarios_actualizados.items():
        carga = cargas_trabajo.get(codigo, 0)
        resultado[codigo] = precio_unitario * carga

    return resultado

### Sumador de diccionarios

In [260]:
from typing import Any, Dict, Union

def sumar_dicts(*dicts: Dict[Any, Any]) -> Dict[Any, Union[int, float]]:
    """
    Suma múltiples diccionarios numéricos.

    Args:
        *dicts: Diccionarios con valores numéricos (int o float).

    Returns:
        Un nuevo diccionario con la suma de valores por clave.

    Raises:
        TypeError: si algún valor no es int ni float o un argumento no es dict.
    """
    result: Dict[Any, Union[int, float]] = {}

    for idx, d in enumerate(dicts, start=1):
        if not isinstance(d, dict):
            raise TypeError(f"El argumento en posición {idx} no es un dict.")
        for key, value in d.items():
            if not isinstance(value, (int, float)):
                raise TypeError(
                    f"El valor para clave '{key}' en el dict {idx} "
                    "no es numérico (int o float)."
                )
            result[key] = result.get(key, 0) + value

    return result

## Carga de datos

### Pickle

In [261]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_cargas_trabajo.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    cargas_trabajo_programadas_anualmente = pickle.load(f)


In [ ]:
#eliminando el total
cargas_trabajo_programadas_anualmente = cargas_trabajo_programadas_anualmente.iloc[:-1]

cargas_trabajo_programadas_anualmente.tail(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
codigo,,,,,,,,,,
MR102,153.068462,328.003846,328.003846,328.003846,328.003846,328.003846,328.003846,328.003846,109.334615,2558.43
MR103,0.000000,5.007500,0.000000,5.007500,5.007500,0.000000,5.007500,0.000000,0.000000,20.03
MR104,5.085714,10.171429,5.085714,10.171429,10.171429,5.085714,10.171429,10.171429,5.085714,71.20
MR201,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR203,22.224000,44.448000,22.224000,44.448000,44.448000,44.448000,44.448000,44.448000,22.224000,333.36
MR204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR206,0.000000,33.910000,33.910000,33.910000,33.910000,33.910000,33.910000,33.910000,0.000000,237.37


### Firebase

In [263]:
## Datos de # 1. Busca el .env en el directorio actual o en cualquiera de los padres
dotenv_path = find_dotenv()
if not dotenv_path:
    raise FileNotFoundError("No se encontró ningún archivo .env en este directorio ni en sus padres.")
load_dotenv(dotenv_path)

# 2. Define el root del proyecto como la carpeta que contiene el .env
project_root = Path(dotenv_path).parent

# 3. Obtén la ruta relativa de las credenciales desde la variable de entorno
rel_cred_path = os.getenv("FIRESTORE_CREDENTIALS")
if not rel_cred_path:
    raise RuntimeError("No existe la variable FIRESTORE_CREDENTIALS en el .env")

# 4. Construye la ruta absoluta al JSON
cred_path = Path(rel_cred_path)
if not cred_path.is_absolute():
    cred_path = (project_root / cred_path).resolve()

if not cred_path.exists():
    raise FileNotFoundError(f"No existe el archivo de credenciales en: {cred_path}")

# 5. Carga las credenciales y crea el cliente de Firestore
credentials = service_account.Credentials.from_service_account_file(str(cred_path))
client = firestore.Client(credentials=credentials, project=credentials.project_id)

# 6. Prueba que funcione
print("Colecciones disponibles:", [c.id for c in client.collections()])

Colecciones disponibles: ['rutinarios']


In [264]:
db=firestore.Client(credentials=credentials, project=credentials.project_id)

#### Cargas de trabajo

In [265]:
cargas_trabajo_contratista=db.collection("rutinarios").document(proyecto).collection("valorizaciones").document(str(mes_en_analisis)).get().to_dict()

pprint.pprint(cargas_trabajo_contratista)

{'cargas_trabajo_contratista_corregido': {'MR101': 0.99,
                                          'MR102': 322.9,
                                          'MR103': 5.01,
                                          'MR104': 10.17,
                                          'MR203': 44.45,
                                          'MR206': 33.91,
                                          'MR301': 3903.83,
                                          'MR701': 3.43},
 'cargas_trabajo_contratista_inicial': {'MR101': 0.99,
                                        'MR102': 328.0,
                                        'MR103': 5.01,
                                        'MR104': 10.17,
                                        'MR203': 44.45,
                                        'MR206': 33.91,
                                        'MR301': 3903.83,
                                        'MR701': 3.43}}


In [266]:
cargas_trabajo_contratista_corregido=cargas_trabajo_contratista['cargas_trabajo_contratista_corregido']

#### Expediente técnico

In [267]:
doc_proyecto_firebase = db.collection("rutinarios").document(proyecto).get().to_dict()
expediente_tecnico_firebase = doc_proyecto_firebase["expediente"]

pprint.pprint(expediente_tecnico_firebase)

{'cargas_trabajo': {'MR101': 7.95,
                    'MR102': 2558.43,
                    'MR103': 20.03,
                    'MR104': 71.2,
                    'MR203': 333.36,
                    'MR206': 237.37,
                    'MR301': 31230.64,
                    'MR401': 15.97,
                    'MR601': 39.73,
                    'MR701': 17.16},
 'codigo_ruta': 'AR-692',
 'coordenadas': {'fin': {'altitud': 4362,
                         'datum': 'WGS84',
                         'hemisferio': 'S',
                         'progresiva': 11920,
                         'x': 254510.31,
                         'y': 8277504.58,
                         'zona': None,
                         'zona_letra': None},
                 'inicio': {'altitud': 3980,
                            'datum': 'WGS84',
                            'hemisferio': 'S',
                            'progresiva': 0,
                            'x': 246384.65,
                            'y': 82800

In [268]:
progresiva_inicio = expediente_tecnico_firebase["coordenadas"][
    "inicio"
]["progresiva"]
progresiva_fin = expediente_tecnico_firebase["coordenadas"][
    "fin"
]["progresiva"]

print(progresiva_inicio)
print(progresiva_fin)

0
11920


#### Precios unitarios

In [269]:
precios_unitarios_expediente=expediente_tecnico_firebase["precios_unitarios"]
pprint.pprint(precios_unitarios_expediente)

{'MR101': 265.3,
 'MR102': 10.27,
 'MR103': 21.0,
 'MR104': 17.5,
 'MR201': 0.44,
 'MR202': 78.5,
 'MR203': 5.25,
 'MR204': 0.44,
 'MR205': 105,
 'MR206': 3.6,
 'MR301': 0.14,
 'MR401': 11.42,
 'MR501': 0.53,
 'MR601': 2.1,
 'MR701': 43.75,
 'MR702': 210}


In [270]:
cargas_trabajo_expediente=expediente_tecnico_firebase["cargas_trabajo"]

In [271]:
costos_expediente=fusionar_diccionarios({
    'carga_trabajo':cargas_trabajo_expediente,
    'precio_unitario':precios_unitarios_expediente
})

pprint.pprint(costos_expediente)

{'MR101': {'carga_trabajo': 7.95, 'precio_unitario': 265.3},
 'MR102': {'carga_trabajo': 2558.43, 'precio_unitario': 10.27},
 'MR103': {'carga_trabajo': 20.03, 'precio_unitario': 21.0},
 'MR104': {'carga_trabajo': 71.2, 'precio_unitario': 17.5},
 'MR203': {'carga_trabajo': 333.36, 'precio_unitario': 5.25},
 'MR206': {'carga_trabajo': 237.37, 'precio_unitario': 3.6},
 'MR301': {'carga_trabajo': 31230.64, 'precio_unitario': 0.14},
 'MR401': {'carga_trabajo': 15.97, 'precio_unitario': 11.42},
 'MR601': {'carga_trabajo': 39.73, 'precio_unitario': 2.1},
 'MR701': {'carga_trabajo': 17.16, 'precio_unitario': 43.75}}


#### Contrato

In [272]:
contrato = doc_proyecto_firebase["contrato"]
pprint.pprint(contrato)

{'contratista': {'razon_social': 'Grupo ARICOL E.I.R.L', 'ruc': 20606988398},
 'denominacion_tramo_convenio': 'EMP. AR-691 CALLA CALLA - YURACCANCHA - '
                                'CUCHUHURI - PALLALLMAYO (KM 11+920)',
 'fecha_inicio': {'anio': 2025, 'dia': 16, 'mes': 4},
 'id_contrato': '006-2025',
 'jefe_mantenimiento': {'apellido': 'Tinta Cáceres',
                        'dni': 0,
                        'nombre': 'Genaro',
                        'titulo': 'Ingeniero'},
 'monto_contrato': 59504.8,
 'numero_cuadrillas': 1,
 'numero_trabajadores': 3,
 'tiempo_ejecucion_dias': 240,
 'tipo_servicio': 'mantenimiento rutinario'}


In [273]:
monto_contrato = contrato["monto_contrato"]
print(monto_contrato)

59504.8


#### Valorizaciones

In [274]:
valorizaciones_ref = db.collection("rutinarios").document(proyecto).collection('valorizaciones')

pprint.pprint(valorizaciones_ref)

In [275]:
docs = valorizaciones_ref.stream()

# Iterar sobre los documentos y acceder a sus datos

valorizaciones={}
for doc in docs:
    print(f'ID del documento: {doc.id}')
    print(f'Datos del documento: {doc.to_dict()}')
    print("-" * 30)

    valorizaciones[doc.id]=doc.to_dict()

pprint.pprint(valorizaciones)

ID del documento: 4
Datos del documento: {'cargas_trabajo_contratista_corregido': {'MR104': 5.09, 'MR203': 22.22, 'MR102': 153.07, 'MR301': 1951.52, 'MR101': 0.66, 'MR401': 5.32}, 'cargas_trabajo_contratista': {'MR104': 5.09, 'MR203': 22.22, 'MR102': 153.07, 'MR301': 1951.52, 'MR101': 0.66, 'MR401': 5.32}, 'avance': {'porcentaje_ejecutado_acumulado_vigente_mes': 0.06246646927783319, 'porcentaje_programado_vigente_mes': 0.0625, 'porcentaje_programado_acumulado_vigente_mes': 0.0625, 'porcentaje_ejecutado_vigente_mes': 0.06246646927783319}}
------------------------------
ID del documento: 5
Datos del documento: {'cargas_trabajo_contratista_inicial': {'MR206': 33.91, 'MR203': 44.45, 'MR301': 3903.83, 'MR101': 0.99, 'MR104': 10.17, 'MR103': 5.01, 'MR102': 328.0, 'MR701': 3.43}, 'cargas_trabajo_contratista_corregido': {'MR206': 33.91, 'MR203': 44.45, 'MR301': 3903.83, 'MR101': 0.99, 'MR104': 10.17, 'MR103': 5.01, 'MR102': 322.9, 'MR701': 3.43}}
------------------------------
{'4': {'avance':

## Cálculos

### Recalculo de precios unitarios

In [276]:
# Crear DataFrame usando pd.DataFrame.from_dict() con orient='index'
df_costos_expediente = pd.DataFrame.from_dict(
    costos_expediente, orient="index"
)

# Resetear el índice para convertir las claves MR en una columna
df_costos_expediente = df_costos_expediente.reset_index().rename(
    columns={"index": "codigo_MR"}
)

In [277]:
# Agregar columna parcial
df_costos_expediente["parcial"] = (
    df_costos_expediente["precio_unitario"]
    * df_costos_expediente["carga_trabajo"]
)
df_costos_expediente.head()

,codigo_MR,carga_trabajo,precio_unitario,parcial
0,MR206,237.37,3.60,854.5320
1,MR104,71.20,17.50,1246.0000
2,MR701,17.16,43.75,750.7500
3,MR301,31230.64,0.14,4372.2896
4,MR401,15.97,11.42,182.3774


In [278]:
costo_directo = df_costos_expediente["parcial"].sum()
print("costo_directo", costo_directo)

costo_directo 38044.3631


In [279]:
costo_total_expediente = calculadora_costo_total(costo_directo)
print(costo_total_expediente["total"])

51626.2


In [280]:
df_costos_expediente["precio_unitario_actualizado"] = (
    df_costos_expediente["precio_unitario"]
    * (monto_contrato / costo_total_expediente["total"])
)

df_costos_expediente["parcial_actualizado"] = (
    df_costos_expediente["precio_unitario_actualizado"]
    * df_costos_expediente["carga_trabajo"]
)

# Calcular el total
costo_directo_actualizado = df_costos_expediente["parcial_actualizado"].sum()

print(f"El costo directo es: {costo_directo_actualizado}")

El costo directo es: 43850.25853913091


In [281]:
df_costos_expediente.head(10)

,codigo_MR,carga_trabajo,precio_unitario,parcial,precio_unitario_actualizado,parcial_actualizado
0,MR206,237.37,3.60,854.5320,4.149391,984.940897
1,MR104,71.20,17.50,1246.0000,20.170650,1436.150265
2,MR701,17.16,43.75,750.7500,50.426624,865.320876
3,MR301,31230.64,0.14,4372.2896,0.161365,5039.538416
4,MR401,15.97,11.42,182.3774,13.162790,210.209752
5,MR102,2558.43,10.27,26275.0761,11.837290,30284.877607
6,MR103,20.03,21.00,420.6300,24.204780,484.821738
7,MR601,39.73,2.10,83.4330,2.420478,96.165590
8,MR101,7.95,265.30,2109.1350,305.787051,2431.007054
9,MR203,333.36,5.25,1750.1400,6.051195,2017.226344


In [282]:
dict_precios_unitarios_actualizados = dict(
    zip(
        df_costos_expediente["codigo_MR"],
        df_costos_expediente["precio_unitario_actualizado"],
    )
)

pprint.pprint(dict_precios_unitarios_actualizados)

{'MR101': 305.78705076104774,
 'MR102': 11.837289903188692,
 'MR103': 24.204779743618552,
 'MR104': 20.170649786348793,
 'MR203': 6.051194935904638,
 'MR206': 4.1493908131917525,
 'MR301': 0.16136519829079038,
 'MR401': 13.162789746291613,
 'MR601': 2.4204779743618556,
 'MR701': 50.42662446587199}


#### Cálculo de pago de acuerdo a cargas de trabajo

In [283]:
pago_costo_directo_parciales_contratista = calcular_costos_unitarios(
    dict_precios_unitarios_actualizados, cargas_trabajo_expediente
)
pprint.pprint(pago_costo_directo_parciales_contratista)

{'MR101': 2431.0070535503296,
 'MR102': 30284.87760701504,
 'MR103': 484.8217382646796,
 'MR104': 1436.150264788034,
 'MR203': 2017.2263438331702,
 'MR206': 984.9408973273263,
 'MR301': 5039.5384163482895,
 'MR401': 210.20975224827708,
 'MR601': 96.16558992139652,
 'MR701': 865.3208758343633}


In [284]:
df_pago_costo_directo_parciales_contratista = pd.DataFrame.from_dict(
    pago_costo_directo_parciales_contratista, orient="index"
)

df_pago_costo_directo_parciales_contratista = (
    df_pago_costo_directo_parciales_contratista.reset_index().rename(
        columns={"index": "codigo_MR"}
    )
)

df_pago_costo_directo_parciales_contratista = (
    df_pago_costo_directo_parciales_contratista.rename(columns={0: "monto_pago"})
)

df_pago_costo_directo_parciales_contratista

,codigo_MR,monto_pago
0,MR206,984.940897
1,MR203,2017.226344
2,MR301,5039.538416
3,MR101,2431.007054
4,MR104,1436.150265
5,MR103,484.821738
6,MR102,30284.877607
7,MR701,865.320876
8,MR601,96.165590
9,MR401,210.209752


In [285]:
sumar_valores = lambda d: sum(d.values())

In [286]:
pago_costo_directo_contratista = sumar_valores(pago_costo_directo_parciales_contratista)
print(pago_costo_directo_contratista)

43850.25853913091


In [287]:
pago_costo_total_contratista = calculadora_costo_total(pago_costo_directo_contratista)
print(pago_costo_total_contratista["total"])

59504.8


#### Validación

In [288]:
diferencia_costos = abs(
    pago_costo_total_contratista["total"]
    - monto_contrato
)

pprint.pprint(
    {
        "diferencia_costos": diferencia_costos,
        "contrato": monto_contrato,
        "costos_unitarios": pago_costo_total_contratista["total"],
    }
)

if diferencia_costos > 1:
    raise ValueError(
        "La valorizacion de las actividades presentadas por el contratista no es coherente con lo programado"
    )

{'contrato': 59504.8, 'costos_unitarios': 59504.8, 'diferencia_costos': 0.0}


#### Cálculo de cargas de trabajo ejecutadas del contratista

##### Mes en curso

In [289]:
cargas_trabajo_contratista_current_month=valorizaciones[str(mes_en_analisis)]['cargas_trabajo_contratista_corregido']
pprint.pprint(cargas_trabajo_contratista_current_month)

{'MR101': 0.99,
 'MR102': 322.9,
 'MR103': 5.01,
 'MR104': 10.17,
 'MR203': 44.45,
 'MR206': 33.91,
 'MR301': 3903.83,
 'MR701': 3.43}


##### Acumulado

In [290]:
cargas_trabajo_ejecutadas_acumuladas={}
for v in valorizaciones:
    if(int(v)<=mes_en_analisis):
        cargas_trabajo_ejecutadas_acumuladas=sumar_dicts(cargas_trabajo_ejecutadas_acumuladas,valorizaciones[v]['cargas_trabajo_contratista_corregido'])

pprint.pprint(cargas_trabajo_ejecutadas_acumuladas)

{'MR101': 1.65,
 'MR102': 475.96999999999997,
 'MR103': 5.01,
 'MR104': 15.26,
 'MR203': 66.67,
 'MR206': 33.91,
 'MR301': 5855.35,
 'MR401': 5.32,
 'MR701': 3.43}


#### Cálculo de cargas de trabajo programadas

##### Mes en curso

In [291]:
cargas_trabajo_programadas_current_month = cargas_trabajo_programadas_anualmente[f"2025-{mes_en_analisis:02d}"].to_dict()
pprint.pprint(cargas_trabajo_programadas_current_month)

{'MR101': 0.99375,
 'MR102': 328.0038461538461,
 'MR103': 5.0075,
 'MR104': 10.17142857142857,
 'MR201': 0.0,
 'MR202': 0.0,
 'MR203': 44.448,
 'MR204': 0.0,
 'MR205': 0.0,
 'MR206': 33.91,
 'MR301': 3903.83,
 'MR401': 0.0,
 'MR501': 0.0,
 'MR601': 0.0,
 'MR701': 3.4320000000000004,
 'MR702': 0.0}


##### Acumulado

In [292]:
cargas_trabajo_programadas_anualmente.drop(columns=['TOTAL'], inplace=True)

In [293]:
# Asumiendo que tu dataframe se llama 'df'
cargas_trabajo_programadas_anualmente_dict = {}

# Iterar por las columnas del dataframe (excluyendo la columna 'codigo' si existe)
for columna in cargas_trabajo_programadas_anualmente.columns:
    if columna != 'codigo':  # Excluir la columna de códigos
        # Extraer el mes de la columna (formato "2025-MM")
        mes = int(columna.split('-')[1])  # Esto convierte "04" a 4, "12" a 12
        
        # Crear el diccionario interno para este mes
        cargas_trabajo_programadas_anualmente_dict[mes] = {}
        
        # Llenar el diccionario con los códigos y valores
        for codigo in cargas_trabajo_programadas_anualmente.index:  # Asumiendo que 'codigo' es el índice
            cargas_trabajo_programadas_anualmente_dict[mes][codigo] = float(cargas_trabajo_programadas_anualmente.loc[codigo, columna])

pprint.pprint(cargas_trabajo_programadas_anualmente_dict)

{4: {'MR101': 0.6625,
     'MR102': 153.06846153846155,
     'MR103': 0.0,
     'MR104': 5.085714285714285,
     'MR201': 0.0,
     'MR202': 0.0,
     'MR203': 22.224,
     'MR204': 0.0,
     'MR205': 0.0,
     'MR206': 0.0,
     'MR301': 1951.915,
     'MR401': 5.323333333333333,
     'MR501': 0.0,
     'MR601': 0.0,
     'MR701': 0.0,
     'MR702': 0.0},
 5: {'MR101': 0.99375,
     'MR102': 328.0038461538461,
     'MR103': 5.0075,
     'MR104': 10.17142857142857,
     'MR201': 0.0,
     'MR202': 0.0,
     'MR203': 44.448,
     'MR204': 0.0,
     'MR205': 0.0,
     'MR206': 33.91,
     'MR301': 3903.83,
     'MR401': 0.0,
     'MR501': 0.0,
     'MR601': 0.0,
     'MR701': 3.4320000000000004,
     'MR702': 0.0},
 6: {'MR101': 0.99375,
     'MR102': 328.0038461538461,
     'MR103': 0.0,
     'MR104': 5.085714285714285,
     'MR201': 0.0,
     'MR202': 0.0,
     'MR203': 22.224,
     'MR204': 0.0,
     'MR205': 0.0,
     'MR206': 33.91,
     'MR301': 3903.83,
     'MR401': 5.32333333333

In [ ]:
cargas_trabajo_programadas_acumuladas={}

for v in cargas_trabajo_programadas_anualmente_dict:
    if(int(v)<=mes_en_analisis):
        cargas_trabajo_programadas_acumuladas=sumar_dicts(cargas_trabajo_programadas_acumuladas,cargas_trabajo_programadas_anualmente_dict[v])

pprint.pprint(cargas_trabajo_programadas_acumuladas)

4
12
5
12
{'MR101': 1.65625,
 'MR102': 481.0723076923076,
 'MR103': 5.0075,
 'MR104': 15.257142857142856,
 'MR201': 0.0,
 'MR202': 0.0,
 'MR203': 66.672,
 'MR204': 0.0,
 'MR205': 0.0,
 'MR206': 33.91,
 'MR301': 5855.745,
 'MR401': 5.323333333333333,
 'MR501': 0.0,
 'MR601': 0.0,
 'MR701': 3.4320000000000004,
 'MR702': 0.0}


#### Avance valorizado ejecutado

##### Mensual

In [308]:
avance_valorizado_ejecutado=calcular_costos_unitarios_v2(dict_precios_unitarios_actualizados=dict_precios_unitarios_actualizados,cargas_trabajo=cargas_trabajo_contratista_current_month)

pprint.pprint(avance_valorizado_ejecutado)

{'MR101': 302.7291802534373,
 'MR102': 3822.2609097396285,
 'MR103': 121.26594651552894,
 'MR104': 205.13550832716723,
 'MR203': 268.97561490096115,
 'MR206': 140.7058424753323,
 'MR301': 629.9423020435362,
 'MR401': 0.0,
 'MR601': 0.0,
 'MR701': 172.96332191794093}


In [309]:
sumar_valores = lambda d: sum(d.values())
costo_directo_avance_valorizado_ejecutado=sumar_valores(avance_valorizado_ejecutado)
print(costo_directo_avance_valorizado_ejecutado)

5663.978626173533


In [310]:
costo_total_avance_valorizado_ejecutado = calculadora_costo_total(costo_directo_avance_valorizado_ejecutado)
print(costo_total_avance_valorizado_ejecutado["total"])

7686.02


##### Acumulado

In [311]:
avance_valorizado_ejecutado_acumulado=calcular_costos_unitarios_v2(dict_precios_unitarios_actualizados=dict_precios_unitarios_actualizados,cargas_trabajo=cargas_trabajo_ejecutadas_acumuladas)

pprint.pprint(avance_valorizado_ejecutado_acumulado)

{'MR101': 504.54863375572876,
 'MR102': 5634.194875220722,
 'MR103': 121.26594651552894,
 'MR104': 307.8041157396826,
 'MR203': 403.43316637676224,
 'MR206': 140.7058424753323,
 'MR301': 944.8497138119795,
 'MR401': 70.02604145027139,
 'MR601': 0.0,
 'MR701': 172.96332191794093}


In [312]:
sumar_valores = lambda d: sum(d.values())
costo_directo_avance_valorizado_ejecutado_acumulado=sumar_valores(avance_valorizado_ejecutado_acumulado)
print(costo_directo_avance_valorizado_ejecutado_acumulado)

8299.791657263948


In [ ]:
costo_total_avance_valorizado_ejecutado_acumulado = calculadora_costo_total(costo_directo_avance_valorizado_ejecutado_acumulado)
print(costo_total_avance_valorizado_ejecutado_acumulado["total"])

11262.82


### Avance valorizado programado

#### Mensual

In [314]:
avance_valorizado_programado=calcular_costos_unitarios_v2(dict_precios_unitarios_actualizados=dict_precios_unitarios_actualizados,cargas_trabajo=cargas_trabajo_programadas_current_month)

pprint.pprint(avance_valorizado_ejecutado)

{'MR101': 302.7291802534373,
 'MR102': 3822.2609097396285,
 'MR103': 121.26594651552894,
 'MR104': 205.13550832716723,
 'MR203': 268.97561490096115,
 'MR206': 140.7058424753323,
 'MR301': 629.9423020435362,
 'MR401': 0.0,
 'MR601': 0.0,
 'MR701': 172.96332191794093}


In [315]:
sumar_valores = lambda d: sum(d.values())
costo_directo_avance_valorizado_programado=sumar_valores(avance_valorizado_programado)
print(costo_directo_avance_valorizado_programado)

5725.5980882819185


In [316]:
costo_total_avance_valorizado_programado = calculadora_costo_total(costo_directo_avance_valorizado_programado)
print(costo_total_avance_valorizado_programado["total"])

7769.64


#### Acumulado

In [317]:
avance_valorizado_programado_acumulado=calcular_costos_unitarios_v2(dict_precios_unitarios_actualizados=dict_precios_unitarios_actualizados,cargas_trabajo=cargas_trabajo_programadas_acumuladas)

pprint.pprint(avance_valorizado_programado_acumulado)

{'MR101': 506.4598028229853,
 'MR102': 5694.592370549837,
 'MR103': 121.2054345661699,
 'MR104': 307.7464853117216,
 'MR203': 403.445268766634,
 'MR206': 140.7058424753323,
 'MR301': 944.9134530653043,
 'MR401': 70.06991741609235,
 'MR601': 0.0,
 'MR701': 173.06417516687267}


In [318]:
sumar_valores = lambda d: sum(d.values())
costo_directo_avance_valorizado_programado_acumulado=sumar_valores(avance_valorizado_programado_acumulado)
print(costo_directo_avance_valorizado_programado_acumulado)

8362.20275014095


In [319]:
costo_total_avance_valorizado_programado_acumulado = calculadora_costo_total(costo_directo_avance_valorizado_programado_acumulado)
print(costo_total_avance_valorizado_programado_acumulado["total"])

11347.51


### Resultados

In [323]:
porcentaje_avance_ejecutado = (
    costo_total_avance_valorizado_ejecutado["total"] / monto_contrato
)
porcentaje_avance_ejecutado_acumulado = (
    costo_total_avance_valorizado_ejecutado_acumulado["total"] / monto_contrato
)
porcentaje_avance_programado = (
    costo_total_avance_valorizado_programado["total"] / monto_contrato
)
porcentaje_avance_programado_acumulado = (
    costo_total_avance_valorizado_programado_acumulado["total"] / monto_contrato
)

print(
    f"{porcentaje_avance_ejecutado=}\n{porcentaje_avance_ejecutado_acumulado=}\n{porcentaje_avance_programado=}\n{porcentaje_avance_programado_acumulado=}"
)

porcentaje_avance_ejecutado=0.1291663865772173
porcentaje_avance_ejecutado_acumulado=0.18927582312687377
porcentaje_avance_programado=0.13057165136257914
porcentaje_avance_programado_acumulado=0.19069906965488498
